#¿Qué día es hoy?

Este cuaderno sirve para generar una notificación en tu ordenador, en esta notificación se aporta información acerca del día de hoy (p.e. si hoy es Lunes de Pascua o el Día del Libro).

Puede ser útil ejecutarlo periódicamente de forma automática.

Es necesario tener conexión a internet, pues la información se obtiene de un portal web.

Es posible que sea neceario instalar las librerías que vamos a importar en este código, estas instalaciones aparecen comentadas.

In [ ]:
#!/usr/bin/python3

Esta línea funciona en caso de ejecutar este script desde la terminal de un sistema Linux, en otros casos es ignorada.

In [38]:
#Instalaciones de librerías, de ser alguna necesaria eliminar símbolo de comentario '#'

#!pip install requests
#!pip install beautifulSoup4
#!pip install notify2

In [ ]:
#Librerías necesarias para obtener la información de internet
import requests
from bs4 import BeautifulSoup

In [ ]:
#Mediante esta primera función se obtiene el texto completo del portal web
def extraer_texto_desde_url(url):
    try:
        # Realizar la solicitud GET a la URL
        response = requests.get(url)

        # Comprobar si la solicitud fue exitosa
        if response.status_code == 200:
            # Obtener el contenido HTML de la página
            html = response.text

            # Parsear el HTML con BeautifulSoup
            soup = BeautifulSoup(html, "html.parser")

            # Encontrar todo el texto en la página
            textorec = soup.get_text(separator="\n")

            # Imprimir el texto
            #print(textorec)

            #guardamos variable
            return textorec
        else:
            print("Error al realizar la solicitud GET")
    except Exception as e:
        print("Ocurrió un error:", e)

url = "https://www.diainternacionalde.com/"
textorec = extraer_texto_desde_url(url)

Las funciones siguientes se encargan de limpiar el texto y darle el formato adecuado

In [ ]:
import re


def extraer_texto_entre_cadenas(texto):
    # Patrón de expresión regular para encontrar el texto específico que buscamos
    patron = r'Hoy(.*?)A un click'
    # Buscar coincidencias en el texto
    coincidencias = re.search(patron, texto, re.DOTALL)
    if coincidencias:
        # Devolver el texto encontrado
        return coincidencias.group(1).strip()
    else:
        return "No se encontraron coincidencias"


# Llamar a la función con el texto de ejemplo
texto_extraido = extraer_texto_entre_cadenas(textorec)
#print(texto_extraido)

In [29]:
def eliminar_filas_vacias(texto):
    # Dividir el texto en líneas
    lineas = texto.split('\n')

    # Eliminar las líneas vacías
    lineas_filtradas = [linea for linea in lineas if linea.strip()]

    # Unir las líneas restantes en un solo texto
    texto_filtrado = '\n'.join(lineas_filtradas)
    #print(texto_filtrado)

    return texto_filtrado

texto_extraido = eliminar_filas_vacias(texto_extraido)

In [32]:
def separar_primera_linea(texto):
    # Dividir el texto en líneas
    lineas = texto.split('\n', 1)

    # Extraer la primera línea y el resto del texto
    primera_linea = lineas[0]
    resto_texto = lineas[1] if len(lineas) > 1 else ''

    return primera_linea, resto_texto


primera_linea, resto_texto = separar_primera_linea(texto_extraido)

#print("Primera línea:", primera_linea)
#print("Resto del texto:")
#print(resto_texto)

In [31]:
def eliminar_lineas_numeradas(texto):
    # Dividir el texto en líneas
    lineas = texto.split('\n')

    # Filtrar las líneas que no comienzan con un número
    lineas_filtradas = [linea for linea in lineas if not re.match(r'^\s*\d', linea)]

    # Unir las líneas filtradas en un solo texto
    texto_filtrado = '\n'.join(lineas_filtradas)

    return texto_filtrado


texto_extraido = eliminar_lineas_numeradas(resto_texto)

#print(texto_extraido)

In [33]:
def agregar_salto_despues_de_se(texto):
    # Dividir el texto en líneas
    lineas = texto.split('\n')

    # Procesar cada línea
    lineas_procesadas = []
    for linea in lineas:
        if re.match(r'^\s*Se', linea):
            # Si la línea comienza con "Se", agregar un salto después de ella
            lineas_procesadas.append(linea)
            lineas_procesadas.append('')  # Agregar línea vacía
        else:
            lineas_procesadas.append(linea)

    # Unir las líneas procesadas en un solo texto
    texto_procesado = '\n'.join(lineas_procesadas)

    return texto_procesado

texto_extraido = agregar_salto_despues_de_se(texto_extraido)

#print(texto_extraido)

La función siguiente sirve para generar la notificación

In [ ]:
import notify2

def enviar_notificacion(titulo, mensaje):
    # Inicializar la librería de notificaciones
    notify2.init("Notificaciones")

    # Crear una nueva notificación
    notificacion = notify2.Notification(titulo, mensaje)

    # Mostrar la notificación
    notificacion.show()

# Ejemplo de uso
enviar_notificacion(primera_linea, texto_extraido)
